In [26]:
import os
import torch
import GPUtil
import numpy as np
from sklearn.model_selection import train_test_split

# Imports de la libreria propia
from vecopsciml.kernels.derivative import DerivativeKernels
from vecopsciml.utils import TensOps

# Imports de las funciones creadas para este programa
from model.FFT import FFTNonlinearModel
from utils.folders import create_folder
from utils.load_data import load_data
from trainers.train import train_loop

from itertools import permutations

In [27]:
from collections import Counter

In [28]:
import matplotlib.pyplot as plt
from vecopsciml.operators.zero_order import Mx, My

In [29]:
# Creamos los paths para las distintas carpetas
ROOT_PATH = r'/home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning'
DATA_PATH = os.path.join(ROOT_PATH, r'data/non_linear_1000_0/non_linear_1000_0.pkl')
RESULTS_FOLDER_PATH = os.path.join(ROOT_PATH, r'results/non_linear')
MODEL_RESULTS_PATH = os.path.join(ROOT_PATH, r'results/non_linear/spectral_model')

# Creamos las carpetas que sean necesarias (si ya están creadas se avisará de ello)
create_folder(RESULTS_FOLDER_PATH)
create_folder(MODEL_RESULTS_PATH)

Folder already exists at: /home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning/results/non_linear
Folder already exists at: /home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning/results/non_linear/spectral_model


In [30]:
# Load dataset
dataset = load_data(DATA_PATH)

Data successfully loaded from: /home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning/data/non_linear_1000_0/non_linear_1000_0.pkl


In [31]:
# Convolutional filters to derivate
dx = dataset['x_step_size']
dy = dataset['y_step_size']
D = DerivativeKernels(dx, dy, 0).grad_kernels_two_dimensions()

In [32]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Using device: {DEVICE}")

Using device: cuda


In [33]:
# Train data splitting in train/test
X = torch.tensor(dataset['X_train'], dtype=torch.float32).unsqueeze(1)
y = torch.tensor(dataset['y_train'], dtype=torch.float32).unsqueeze(1)
K = torch.tensor(dataset['k_train'], dtype=torch.float32).unsqueeze(1)
f = torch.tensor(dataset['f_train'], dtype=torch.float32).unsqueeze(1)

X_train, X_test, y_train, y_test, K_train, K_test, f_train, f_test = train_test_split(X, y, K, f, test_size=0.3, random_state=42)

# Data processing and adequacy with our TensOps library
X_train = X_train.to(DEVICE)
X_test = X_test.to(DEVICE)

y_train = TensOps(y_train.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)
y_test = TensOps(y_test.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)

K_train = TensOps(K_train.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)
K_test = TensOps(K_test.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)

f_train = TensOps(f_train.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)
f_test = TensOps(f_test.to(DEVICE).requires_grad_(True), space_dimension=2, contravariance=0, covariance=0)

# Loading and processing validation data
X_val = torch.tensor(dataset['X_val'], dtype=torch.float32).unsqueeze(1)
y_val = TensOps(torch.tensor(dataset['y_val'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, contravariance=0, covariance=0)
K_val = TensOps(torch.tensor(dataset['k_val'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, contravariance=0, covariance=0)
f_val = TensOps(torch.tensor(dataset['f_val'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, contravariance=0, covariance=0)

In [34]:
num_modes = 20  # número total de funciones base deseadas
device = y_train.values[0].device

X_mesh = torch.tensor(dataset['X_mesh'], device=device)
Y_mesh = torch.tensor(dataset['Y_mesh'], device=device)

deg2rad = torch.pi / 180
deg2rad = 1

base = []
mode_idx = 0
max_order = 2 * int(torch.sqrt(torch.tensor(num_modes))) + 1

max_m = max_n = int(torch.sqrt(torch.tensor(num_modes))) + 1

for s in range(max_order):  # s = m + n
    for m in range(s + 1):
        n = s - m

        if mode_idx == 0:
            base.append(torch.ones_like(X_mesh))
            mode_idx += 1
            if mode_idx >= num_modes:
                break

        if m > 0 and n > 0:
            # A_mn
            base.append(torch.cos(2 * torch.pi * m * deg2rad * X_mesh / (max_m - 1) + 2 * torch.pi * n * deg2rad * Y_mesh / (max_n - 1)))
            mode_idx += 1
            if mode_idx >= num_modes:
                break

            # B_mn
            base.append(torch.sin(2 * torch.pi * m * deg2rad * X_mesh / (max_m - 1) + 2 * torch.pi * n * deg2rad * Y_mesh / (max_n - 1)))
            mode_idx += 1
            if mode_idx >= num_modes:
                break

    if mode_idx >= num_modes:
        break

base = torch.stack(base, dim=0)

In [36]:
# Predictive network architecture
input_shape = X_train[0].shape
predictive_layers = [20, 10, num_modes]
predictive_output = y_train.values[0].shape

# Explanatory network architecture
explanatory_input = Mx(My(y_train)).values[0].shape
explanatory_layers = [10, 10]
explanatory_output = Mx(My(f_train)).values[0].shape

# Other parameters 
n_filters_explanatory = 5

In [37]:
# Load model and the optimizer
model = FFTNonlinearModel(input_shape, predictive_layers, base, predictive_output, explanatory_input, explanatory_layers, explanatory_output, n_filters_explanatory, DEVICE).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=3e-3)

# Parametros de entrenamiento
start_epoch = 0
n_epochs = 110000

batch_size = 64
n_checkpoints = 5

train_loop(model, optimizer, X_train, y_train, f_train, X_test, y_test, f_test,
        D,  n_checkpoints, start_epoch=start_epoch, n_epochs=n_epochs, batch_size=batch_size, 
        model_results_path=MODEL_RESULTS_PATH, device=DEVICE)

Starting training from scratch.
Epoch 0, Train loss: 1.229e+08, Test loss: 1.827e+08, MSE(e): 1.063e+01, MSE(pi1): 1.390e+03, MSE(pi2): 9.324e+00, MSE(pi3): 2.719e+01
Epoch 100, Train loss: 1.180e+06, Test loss: 1.853e+06, MSE(e): 1.141e-01, MSE(pi1): 2.015e+00, MSE(pi2): 9.504e-02, MSE(pi3): 1.862e-01
Epoch 200, Train loss: 4.175e+05, Test loss: 5.530e+05, MSE(e): 3.784e-02, MSE(pi1): 1.976e+00, MSE(pi2): 3.205e-02, MSE(pi3): 1.925e-01
Epoch 300, Train loss: 2.728e+05, Test loss: 3.577e+05, MSE(e): 2.341e-02, MSE(pi1): 1.959e+00, MSE(pi2): 1.807e-02, MSE(pi3): 1.909e-01
Epoch 400, Train loss: 1.936e+05, Test loss: 2.076e+05, MSE(e): 1.563e-02, MSE(pi1): 1.886e+00, MSE(pi2): 1.193e-02, MSE(pi3): 1.845e-01
Epoch 500, Train loss: 1.482e+05, Test loss: 1.631e+05, MSE(e): 1.158e-02, MSE(pi1): 1.631e+00, MSE(pi2): 8.186e-03, MSE(pi3): 1.617e-01
Epoch 600, Train loss: 1.103e+05, Test loss: 1.254e+05, MSE(e): 9.077e-03, MSE(pi1): 1.013e+00, MSE(pi2): 6.675e-03, MSE(pi3): 9.359e-02
Epoch 700, 

In [38]:
# Parametros de entrenamiento
start_epoch = 100000
n_epochs = 150000

batch_size = 64 
n_checkpoints = 10

second_lr = 3e-4


train_loop(model, optimizer, X_train, y_train, f_train, X_test, y_test, f_test,
        D,  n_checkpoints, start_epoch=start_epoch, n_epochs=n_epochs, batch_size=batch_size, 
        model_results_path=MODEL_RESULTS_PATH, device=DEVICE, new_lr=second_lr)

Starting training from a checkpoint. Epoch 100000.


FileNotFoundError: [Errno 2] No such file or directory: '/home/rmunoz/Escritorio/rmunozTMELab/Physically-Guided-Machine-Learning/results/non_linear/spectral_model/epoch_100000.pth'